# Estimação do potencial de trabalho remoto no Brasil em 2019 

A estimação será feita usando, primeiramente, a base da PNAD Contínua suplementar de 2019 e, depois, a PNAD COVID

Foi adotado a hipótese de que a perda de potencial de trabalho remoto devido à falta de infraestrutura é a mesma na PNAD-COVID que na PNAD Contínua de 2019.

Trabalhar com as seguintes variáveis da PNAD Contínua: 
- V4010, que classifica as diversas ocupações, 
- a unidade da federação (UF), 
- gênero (V2007), 
- raça (V2010), 
- renda em todos os trabalhos (VD4019), 
- Setor Público (V4013), 
- educação (VD3004), 
- formal/informal (combinação da variável VD4009 com a variável V4019) e 
- o peso amostral (V1032).

Não esquecer de filtrar a base para os registros que a 'dummy_infra' igual à 1.



In [1]:
#pacotes 
import pandas as pd
import numpy as np
import datetime as dt

In [13]:
#Importação dos dados da PNAD Contínua 2019 (suplementar)

#determina qual diretório usar para cada sistema operacional.
import os
if os.name == 'nt': #windows
    diretorio = 'C:\\Users\\est.matheus.brasil\\Documents\\Microdados PNAD Contínua\\suplementar\\'

else: #linux(ou mac)
    diretorio = '/home/matheus/Microdados PNAD/feather/'

filtro_pnadc = ['Trimestre','V4010', 'UF', 'V2007', 'V2010', 'VD4019', 'V4013', 'VD3004', 'VD4009', 'V4019', 'V1032', 'dummy_infra']

pnadc_19 = pd.read_feather(fr'{diretorio}PNADC_2019_editado.feather', columns=filtro_pnadc)



#Importação dos códigos de ocupação traduzidos

ocupacoes = pd.read_excel('../Anexos/ocupacoes_traduzidas.xlsx')

In [16]:
#Importação dos dados da PNAD COVID

#determina qual diretório usar para cada sistema operacional.
import os
if os.name == 'nt': #windows
    diretorio = 'C:\\Users\\est.matheus.brasil\\Documents\\Microdados PNAD Contínua\\feather_covid\\'

else: #linux(ou mac)
    diretorio = '/home/matheus/Microdados PNAD/feather/'

#Leitura de todas as bases com as variáveis selecionadas e criação de variáveis dinamicamente.

filtro_pnad_covid = ['Trimestre','V4010', 'UF', 'V2007', 'V2010', 'VD4019', 'V4013', 'VD3004', 'VD4009', 'V4019', 'V1032', 'dummy_infra']


for i in range(5, 12):
    if i < 10:
        mes = '0' + str(i)
    else:
        mes = i
    print(f'Importando PNAD COVID {mes} 2020')
    globals()[f'pnad_covid_{mes}2020'] = pd.read_feather(fr'{diretorio}PNAD_COVID_{mes}2020.feather', columns=filtro_pnad_covid)

print('Fim da importação')

Importando PNAD COVID 05 2020


ArrowInvalid: Field named Trimestre is not found

In [18]:
pnad_covid_112020['C007C'].value_counts()

Agricultor, criador de animais, pescador, silvicultor e jardineiro                                14975
Outros                                                                                            13752
Pedreiro, servente de pedreiro, pintor, eletricista, marceneiro                                   10469
Empregado doméstico, diarista, cozinheiro (em domicílios particulares),                            7514
Outra profissão de nível superior (advogado, engenheiro, contador, jornalista etc.)                7447
Professor da educação infantil, de ensino fundamental, médio ou superior,                          6319
Balconista, vendedor de loja                                                                       6017
Auxiliar de escritório, escriturário                                                               5832
Outro técnico ou profissional de nível médio                                                       5675
Comerciante (dono do bar, da loja etc.)                         

In [3]:
ocupacoes.head(3)

,code_ONET,Title_ONET,teleworkable,Cod,Titulação
0,11-1011,Chief Executives,1,1,DIRETORES E GERENTES
1,11-1011,Chief Executives,1,1,DIRETORES E GERENTES
2,11-1011,Chief Executives,1,1,DIRETORES E GERENTES


In [4]:
pnadc_19.head(3)

,Trimestre,V4010,UF,V2007,V2010,VD4019,V4013,VD3004,VD4009,V4019,V1032,dummy_infra
0,1,3221,Rondônia,Mulher,Parda,1583.0,86001,Superior completo,Empregado no setor privado com carteira de tra...,NaN,149.43857,0
1,1,None,Rondônia,Homem,Parda,NaN,None,Fundamental completo ou equivalente,NaN,NaN,149.43857,0
2,1,None,Rondônia,Mulher,Branca,NaN,None,Médio completo ou equivalente,NaN,NaN,149.43857,0


### Junção dos códigos de ocupação com os dados da PNAD Contínua 2019

In [5]:
pnadc_19['V4010'].value_counts().sort_index()

0000     33
0110    116
0210    573
0411     78
0412    715
       ... 
9621    457
9622     25
9623     36
9624      9
9629    507
Name: V4010, Length: 427, dtype: int64

In [6]:
#Coluna com o código da ocupação Major das ocupações traduzidas
pnadc_19['COD_OCUP_MAJ'] = pnadc_19['V4010'].astype(str).str[:1]

In [7]:
ocupacoes['Cod'] = ocupacoes['Cod'].astype(str)

In [8]:
pnadc_19_remoto = pd.merge(pnadc_19, ocupacoes, left_on='COD_OCUP_MAJ', right_on='Cod', how='left')

In [9]:
pnadc_19_remoto.head(2)

,Trimestre,V4010,UF,V2007,V2010,VD4019,V4013,VD3004,VD4009,V4019,V1032,dummy_infra,COD_OCUP_MAJ,code_ONET,Title_ONET,teleworkable,Cod,Titulação
0,1,3221,Rondônia,Mulher,Parda,1583.0,86001,Superior completo,Empregado no setor privado com carteira de tra...,NaN,149.43857,0,3,11-9141,"Property, Real Estate, and Community Associati...",1.0,3,TÉCNICOS E PROFISSIONAIS DE NÍVEL MÉDIO
1,1,3221,Rondônia,Mulher,Parda,1583.0,86001,Superior completo,Empregado no setor privado com carteira de tra...,NaN,149.43857,0,3,13-1021,"Buyers and Purchasing Agents, Farm Products",1.0,3,TÉCNICOS E PROFISSIONAIS DE NÍVEL MÉDIO


Até aqui, a PNAD Contínua com a dummy de infraestrutura e a base de ocupações traduzidas da O*NET para COD foram e unidas no dataframe <code>pnadc_19_remoto</code>.

### Junção dos códigos de ocupação com os dados da PNAD Contínua 2019

### Ajuste do trabalho remoto potencial 

O ajuste deve ser realizada através da seguinte equação:

$POT^K_{PNAD-COVID-INFRA} = POT^K_{PNAD-COVID} \times \frac{POT^K_{PNAD-INFRA}}{POT^K_{PNAD}}$

Em que:

- $POT^K_{PNAD-COVID-INFRA}$ é o trabalho remoto potencial da PNAD-COVID ajustado pela infraestrutura,

- $POT^K_{PNAD-COVID}$ é o potencial de trabalho remoto da PNAD-COVID sem o ajuste da infraestrutura,

- $POT^K_{PNAD-INFRA}$ é o potencial de trabalho remoto da PNAD Contínua de 2019 com ajuste de infraestrutura e 

- $POT^K_{PNAD}$ é o potencial de trabalho remoto calculado com base na PNAD Contínua de 2019 sem o ajuste de infraestrutura.


In [12]:
pnadc_19_remoto.groupby('Titulação')[['V4010']].count()

,V4010
Titulação,
DIRETORES E GERENTES,871542
OCUPAÇÕES ELEMENTARES,1950088
OPERADORES DE INSTALAÇÕES E MÁQUINAS E MONTADORES,1557504
OUTROS,279384
PROFISSIONAIS DAS CIÊNCIAS E INTELECTUAIS,5116839
TRABALHADORES DE APOIO ADMINISTRATIVO,502793
"TRABALHADORES DOS SERVIÇOS, VENDEDORES DOS COMÉRCIOS E MERCADOS",2828732
"TRABALHADORES QUALIFICADOS DA AGROPECUÁRIA, FLORESTAIS, DA CAÇA E DA PESCA",1651179
"TRABALHADORES QUALIFICADOS, OPERÁRIOS E ARTESÃOS DA CONSTRUÇÃO, DAS ARTES MECÂNICAS E OUTROS OFÍCIOS",3897330


In [11]:
pnadc_19_remoto[pnadc_19_remoto['dummy_infra'] == 1].groupby('Titulação')['V4010'].count() / pnadc_19_remoto.groupby('Titulação')['V4010'].count()

Titulação
DIRETORES E GERENTES                                                                                    0.769987
OCUPAÇÕES ELEMENTARES                                                                                   0.205841
OPERADORES DE INSTALAÇÕES E MÁQUINAS E MONTADORES                                                       0.386552
OUTROS                                                                                                  0.755262
PROFISSIONAIS DAS CIÊNCIAS E INTELECTUAIS                                                               0.841648
TRABALHADORES DE APOIO ADMINISTRATIVO                                                                   0.643094
TRABALHADORES DOS SERVIÇOS, VENDEDORES DOS COMÉRCIOS E MERCADOS                                         0.429938
TRABALHADORES QUALIFICADOS DA AGROPECUÁRIA, FLORESTAIS, DA CAÇA E DA PESCA                              0.175733
TRABALHADORES QUALIFICADOS, OPERÁRIOS E ARTESÃOS DA CONSTRUÇÃO, DAS ARTES MECÂNICAS E 